In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sb
import scipy.stats as stat
%matplotlib inline
import random
from geopy.distance import vincenty

In [2]:
bikeshare = pd.read_csv('bikeshare_reduced.csv', 
                        parse_dates=['Start date', 'End date', 'start_date_short', 'end_date_short'])
bikeshare.drop('Unnamed: 0', 1, inplace=True)

lat_lon = pd.read_csv('bikeshare_locations.csv')
print(lat_lon.head())

    OBJECTID  ID                                     ADDRESS  TERMINAL_NUMBER  \
0  173869668   1                         Eads St & 15th St S            31000   
1  173869669   2                             18th & Eads St.            31001   
2  173869670   3                           20th & Crystal Dr            31002   
3  173869671   4                           15th & Crystal Dr            31003   
4  173869672   5  Aurora Hills Community Ctr/18th & Hayes St            31004   

    LATITUDE  LONGITUDE INSTALLED LOCKED  INSTALL_DATE  REMOVAL_DATE  \
0  38.858971 -77.053230       YES     NO           NaN           NaN   
1  38.857250 -77.053320       YES     NO           NaN           NaN   
2  38.856425 -77.049232       YES     NO           NaN           NaN   
3  38.860170 -77.049593       YES     NO           NaN           NaN   
4  38.857866 -77.059490       YES     NO           NaN           NaN   

  TEMPORARY_INSTALL  NUMBER_OF_BIKES  NUMBER_OF_EMPTY_DOCKS            X  \
0   

In [3]:
lat_lon = lat_lon[['ADDRESS', 'LATITUDE', 'LONGITUDE']]

In [4]:
lat_lon.head()

,ADDRESS,LATITUDE,LONGITUDE
0,Eads St & 15th St S,38.858971,-77.053230
1,18th & Eads St.,38.857250,-77.053320
2,20th & Crystal Dr,38.856425,-77.049232
3,15th & Crystal Dr,38.860170,-77.049593
4,Aurora Hills Community Ctr/18th & Hayes St,38.857866,-77.059490


In [5]:
bikeshare.head()

,Duration,Start date,End date,Start station,End station,Member Type,start_date_short,end_date_short,time_diff,season,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,0h 1min. 50sec.,2011-03-31 23:58:00,2011-04-01 00:00:00,14th & Harvard St NW (31105),16th & Harvard St NW (31103),Registered,2011-03-31,2011-04-01,1.833,2,3,0,4,1,3,0.268333,0.257575,0.918333,0.217646
1,0h 16min. 21sec.,2011-03-31 23:52:00,2011-04-01 00:08:00,19th & L St NW (31224),7th & Water St SW / SW Waterfront (31609),Casual,2011-03-31,2011-04-01,16.350,2,3,0,4,1,3,0.268333,0.257575,0.918333,0.217646
2,0h 3min. 19sec.,2011-03-31 23:47:00,2011-03-31 23:50:00,Lincoln Park / 13th & East Capitol St NE (31619),13th & H St NE (31611),Registered,2011-03-31,2011-03-31,3.317,2,3,0,4,1,3,0.268333,0.257575,0.918333,0.217646
3,0h 5min. 44sec.,2011-03-31 23:45:00,2011-03-31 23:50:00,14th & R St NW (31202),Massachusetts Ave & Dupont Circle NW (31200),Registered,2011-03-31,2011-03-31,5.733,2,3,0,4,1,3,0.268333,0.257575,0.918333,0.217646
4,0h 8min. 42sec.,2011-03-31 23:34:00,2011-03-31 23:43:00,Park Rd & Holmead Pl NW (31602),15th & P St NW (31201),Registered,2011-03-31,2011-03-31,8.700,2,3,0,4,1,3,0.268333,0.257575,0.918333,0.217646


In [6]:
small = bikeshare.copy()

In [7]:
small['start_station'], small['start_station_code'] = small['Start station'].str.split("(", 1).str

In [8]:
small['start_station_code'] = small['start_station_code'].str.replace(')', '')

In [9]:
small['start_station'] = small['start_station'].str.strip()

In [10]:
sum(small['start_station_code'].isnull())

353818

In [11]:
len(small['Start station'].unique())

296

In [12]:
len(lat_lon)

485

In [13]:
small['end_station'], small['end_station_code'] = small['End station'].str.split("(", 1).str

In [14]:
small['end_station_code'] = small['end_station_code'].str.replace(')', '')

In [15]:
small['end_station'] = small['end_station'].str.strip()

In [16]:
small.head()

,Duration,Start date,End date,Start station,End station,Member Type,start_date_short,end_date_short,time_diff,season,...,workingday,weathersit,temp,atemp,hum,windspeed,start_station,start_station_code,end_station,end_station_code
0,0h 1min. 50sec.,2011-03-31 23:58:00,2011-04-01 00:00:00,14th & Harvard St NW (31105),16th & Harvard St NW (31103),Registered,2011-03-31,2011-04-01,1.833,2,...,1,3,0.268333,0.257575,0.918333,0.217646,14th & Harvard St NW,31105,16th & Harvard St NW,31103
1,0h 16min. 21sec.,2011-03-31 23:52:00,2011-04-01 00:08:00,19th & L St NW (31224),7th & Water St SW / SW Waterfront (31609),Casual,2011-03-31,2011-04-01,16.350,2,...,1,3,0.268333,0.257575,0.918333,0.217646,19th & L St NW,31224,7th & Water St SW / SW Waterfront,31609
2,0h 3min. 19sec.,2011-03-31 23:47:00,2011-03-31 23:50:00,Lincoln Park / 13th & East Capitol St NE (31619),13th & H St NE (31611),Registered,2011-03-31,2011-03-31,3.317,2,...,1,3,0.268333,0.257575,0.918333,0.217646,Lincoln Park / 13th & East Capitol St NE,31619,13th & H St NE,31611
3,0h 5min. 44sec.,2011-03-31 23:45:00,2011-03-31 23:50:00,14th & R St NW (31202),Massachusetts Ave & Dupont Circle NW (31200),Registered,2011-03-31,2011-03-31,5.733,2,...,1,3,0.268333,0.257575,0.918333,0.217646,14th & R St NW,31202,Massachusetts Ave & Dupont Circle NW,31200
4,0h 8min. 42sec.,2011-03-31 23:34:00,2011-03-31 23:43:00,Park Rd & Holmead Pl NW (31602),15th & P St NW (31201),Registered,2011-03-31,2011-03-31,8.700,2,...,1,3,0.268333,0.257575,0.918333,0.217646,Park Rd & Holmead Pl NW,31602,15th & P St NW,31201


In [17]:
columns = list(small.columns)
columns

['Duration',
 'Start date',
 'End date',
 'Start station',
 'End station',
 'Member Type',
 'start_date_short',
 'end_date_short',
 'time_diff',
 'season',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'start_station',
 'start_station_code',
 'end_station',
 'end_station_code']

In [18]:
list_cols = ['Start date',
 'End date',
 'start_station',
 'start_station_code',
 'end_station',
 'end_station_code',
 'Member Type',
 'start_date_short',
 'end_date_short',
 'time_diff',
 'season',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed']

In [19]:
small = small[list_cols]

In [20]:
small.shape

(1538148, 20)

In [21]:
merged_data = pd.merge(small, lat_lon, how='left', left_on='start_station', right_on='ADDRESS')

In [22]:
merged_data.shape

(1538148, 23)

In [23]:
merged_data.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,weekday,workingday,weathersit,temp,atemp,hum,windspeed,ADDRESS,LATITUDE,LONGITUDE
0,2011-03-31 23:58:00,2011-04-01 00:00:00,14th & Harvard St NW,31105,16th & Harvard St NW,31103,Registered,2011-03-31,2011-04-01,1.833,...,4,1,3,0.268333,0.257575,0.918333,0.217646,14th & Harvard St NW,38.926800,-77.032200
1,2011-03-31 23:52:00,2011-04-01 00:08:00,19th & L St NW,31224,7th & Water St SW / SW Waterfront,31609,Casual,2011-03-31,2011-04-01,16.350,...,4,1,3,0.268333,0.257575,0.918333,0.217646,19th & L St NW,38.903407,-77.043648
2,2011-03-31 23:47:00,2011-03-31 23:50:00,Lincoln Park / 13th & East Capitol St NE,31619,13th & H St NE,31611,Registered,2011-03-31,2011-03-31,3.317,...,4,1,3,0.268333,0.257575,0.918333,0.217646,Lincoln Park / 13th & East Capitol St NE,38.890461,-76.988355
3,2011-03-31 23:45:00,2011-03-31 23:50:00,14th & R St NW,31202,Massachusetts Ave & Dupont Circle NW,31200,Registered,2011-03-31,2011-03-31,5.733,...,4,1,3,0.268333,0.257575,0.918333,0.217646,14th & R St NW,38.912939,-77.032085
4,2011-03-31 23:34:00,2011-03-31 23:43:00,Park Rd & Holmead Pl NW,31602,15th & P St NW,31201,Registered,2011-03-31,2011-03-31,8.700,...,4,1,3,0.268333,0.257575,0.918333,0.217646,Park Rd & Holmead Pl NW,38.930800,-77.031500


In [24]:
merged_data = merged_data[merged_data['LATITUDE'].notnull()]

In [25]:
sum(merged_data['LONGITUDE'].isnull())

0

In [26]:
merged_data = merged_data.rename(columns={'LATITUDE': 'start_lat', 'LONGITUDE': 'start_lon'})

In [27]:
merged_data.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,weekday,workingday,weathersit,temp,atemp,hum,windspeed,ADDRESS,start_lat,start_lon
0,2011-03-31 23:58:00,2011-04-01 00:00:00,14th & Harvard St NW,31105,16th & Harvard St NW,31103,Registered,2011-03-31,2011-04-01,1.833,...,4,1,3,0.268333,0.257575,0.918333,0.217646,14th & Harvard St NW,38.926800,-77.032200
1,2011-03-31 23:52:00,2011-04-01 00:08:00,19th & L St NW,31224,7th & Water St SW / SW Waterfront,31609,Casual,2011-03-31,2011-04-01,16.350,...,4,1,3,0.268333,0.257575,0.918333,0.217646,19th & L St NW,38.903407,-77.043648
2,2011-03-31 23:47:00,2011-03-31 23:50:00,Lincoln Park / 13th & East Capitol St NE,31619,13th & H St NE,31611,Registered,2011-03-31,2011-03-31,3.317,...,4,1,3,0.268333,0.257575,0.918333,0.217646,Lincoln Park / 13th & East Capitol St NE,38.890461,-76.988355
3,2011-03-31 23:45:00,2011-03-31 23:50:00,14th & R St NW,31202,Massachusetts Ave & Dupont Circle NW,31200,Registered,2011-03-31,2011-03-31,5.733,...,4,1,3,0.268333,0.257575,0.918333,0.217646,14th & R St NW,38.912939,-77.032085
4,2011-03-31 23:34:00,2011-03-31 23:43:00,Park Rd & Holmead Pl NW,31602,15th & P St NW,31201,Registered,2011-03-31,2011-03-31,8.700,...,4,1,3,0.268333,0.257575,0.918333,0.217646,Park Rd & Holmead Pl NW,38.930800,-77.031500


In [28]:
full_merge = pd.merge(merged_data, lat_lon, how='left', left_on='end_station', right_on='ADDRESS')

In [29]:
full_merge.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,temp,atemp,hum,windspeed,ADDRESS_x,start_lat,start_lon,ADDRESS_y,LATITUDE,LONGITUDE
0,2011-03-31 23:58:00,2011-04-01 00:00:00,14th & Harvard St NW,31105,16th & Harvard St NW,31103,Registered,2011-03-31,2011-04-01,1.833,...,0.268333,0.257575,0.918333,0.217646,14th & Harvard St NW,38.926800,-77.032200,16th & Harvard St NW,38.926088,-77.036536
1,2011-03-31 23:52:00,2011-04-01 00:08:00,19th & L St NW,31224,7th & Water St SW / SW Waterfront,31609,Casual,2011-03-31,2011-04-01,16.350,...,0.268333,0.257575,0.918333,0.217646,19th & L St NW,38.903407,-77.043648,NaN,NaN,NaN
2,2011-03-31 23:47:00,2011-03-31 23:50:00,Lincoln Park / 13th & East Capitol St NE,31619,13th & H St NE,31611,Registered,2011-03-31,2011-03-31,3.317,...,0.268333,0.257575,0.918333,0.217646,Lincoln Park / 13th & East Capitol St NE,38.890461,-76.988355,13th & H St NE,38.900022,-76.988422
3,2011-03-31 23:45:00,2011-03-31 23:50:00,14th & R St NW,31202,Massachusetts Ave & Dupont Circle NW,31200,Registered,2011-03-31,2011-03-31,5.733,...,0.268333,0.257575,0.918333,0.217646,14th & R St NW,38.912939,-77.032085,Massachusetts Ave & Dupont Circle NW,38.910100,-77.044400
4,2011-03-31 23:34:00,2011-03-31 23:43:00,Park Rd & Holmead Pl NW,31602,15th & P St NW,31201,Registered,2011-03-31,2011-03-31,8.700,...,0.268333,0.257575,0.918333,0.217646,Park Rd & Holmead Pl NW,38.930800,-77.031500,15th & P St NW,38.909850,-77.034438


In [30]:
sum(full_merge['LONGITUDE'].isnull())

144798

In [31]:
full_merge = full_merge[full_merge['LATITUDE'].notnull()]

In [32]:
full_merge.shape

(1219847, 26)

In [33]:
full_merge = full_merge.rename(columns={'LATITUDE': 'end_lat', 'LONGITUDE': 'end_lon'})

In [34]:
full_merge.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,temp,atemp,hum,windspeed,ADDRESS_x,start_lat,start_lon,ADDRESS_y,end_lat,end_lon
0,2011-03-31 23:58:00,2011-04-01 00:00:00,14th & Harvard St NW,31105,16th & Harvard St NW,31103,Registered,2011-03-31,2011-04-01,1.833,...,0.268333,0.257575,0.918333,0.217646,14th & Harvard St NW,38.926800,-77.032200,16th & Harvard St NW,38.926088,-77.036536
2,2011-03-31 23:47:00,2011-03-31 23:50:00,Lincoln Park / 13th & East Capitol St NE,31619,13th & H St NE,31611,Registered,2011-03-31,2011-03-31,3.317,...,0.268333,0.257575,0.918333,0.217646,Lincoln Park / 13th & East Capitol St NE,38.890461,-76.988355,13th & H St NE,38.900022,-76.988422
3,2011-03-31 23:45:00,2011-03-31 23:50:00,14th & R St NW,31202,Massachusetts Ave & Dupont Circle NW,31200,Registered,2011-03-31,2011-03-31,5.733,...,0.268333,0.257575,0.918333,0.217646,14th & R St NW,38.912939,-77.032085,Massachusetts Ave & Dupont Circle NW,38.910100,-77.044400
4,2011-03-31 23:34:00,2011-03-31 23:43:00,Park Rd & Holmead Pl NW,31602,15th & P St NW,31201,Registered,2011-03-31,2011-03-31,8.700,...,0.268333,0.257575,0.918333,0.217646,Park Rd & Holmead Pl NW,38.930800,-77.031500,15th & P St NW,38.909850,-77.034438
5,2011-03-31 23:31:00,2011-03-31 23:39:00,37th & O St NW / Georgetown University,31236,21st & M St NW,31212,Registered,2011-03-31,2011-03-31,7.850,...,0.268333,0.257575,0.918333,0.217646,37th & O St NW / Georgetown University,38.907740,-77.071652,21st & M St NW,38.905340,-77.046774


In [35]:
list(full_merge.columns)

['Start date',
 'End date',
 'start_station',
 'start_station_code',
 'end_station',
 'end_station_code',
 'Member Type',
 'start_date_short',
 'end_date_short',
 'time_diff',
 'season',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'ADDRESS_x',
 'start_lat',
 'start_lon',
 'ADDRESS_y',
 'end_lat',
 'end_lon']

In [36]:
new_list = ['Start date',
 'End date',
 'start_station',
 'start_station_code',
 'end_station',
 'end_station_code',
 'Member Type',
 'start_date_short',
 'end_date_short',
 'time_diff',
 'season',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'start_lat',
 'start_lon',
 'end_lat',
 'end_lon']

In [37]:
full_merge = full_merge[new_list]

In [38]:
full_merge['lat_lon_start'] = list(zip(full_merge.start_lat, full_merge.start_lon))

In [39]:
full_merge['lat_lon_end'] = list(zip(full_merge.end_lat, full_merge.end_lon))

In [40]:
full_merge.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,temp,atemp,hum,windspeed,start_lat,start_lon,end_lat,end_lon,lat_lon_start,lat_lon_end
0,2011-03-31 23:58:00,2011-04-01 00:00:00,14th & Harvard St NW,31105,16th & Harvard St NW,31103,Registered,2011-03-31,2011-04-01,1.833,...,0.268333,0.257575,0.918333,0.217646,38.926800,-77.032200,38.926088,-77.036536,"(38.9268, -77.0322)","(38.926088, -77.036536)"
2,2011-03-31 23:47:00,2011-03-31 23:50:00,Lincoln Park / 13th & East Capitol St NE,31619,13th & H St NE,31611,Registered,2011-03-31,2011-03-31,3.317,...,0.268333,0.257575,0.918333,0.217646,38.890461,-76.988355,38.900022,-76.988422,"(38.890461, -76.988355)","(38.900022, -76.988422)"
3,2011-03-31 23:45:00,2011-03-31 23:50:00,14th & R St NW,31202,Massachusetts Ave & Dupont Circle NW,31200,Registered,2011-03-31,2011-03-31,5.733,...,0.268333,0.257575,0.918333,0.217646,38.912939,-77.032085,38.910100,-77.044400,"(38.912939, -77.032085)","(38.9101, -77.0444)"
4,2011-03-31 23:34:00,2011-03-31 23:43:00,Park Rd & Holmead Pl NW,31602,15th & P St NW,31201,Registered,2011-03-31,2011-03-31,8.700,...,0.268333,0.257575,0.918333,0.217646,38.930800,-77.031500,38.909850,-77.034438,"(38.9308, -77.0315)","(38.90985, -77.034438)"
5,2011-03-31 23:31:00,2011-03-31 23:39:00,37th & O St NW / Georgetown University,31236,21st & M St NW,31212,Registered,2011-03-31,2011-03-31,7.850,...,0.268333,0.257575,0.918333,0.217646,38.907740,-77.071652,38.905340,-77.046774,"(38.90774, -77.071652)","(38.90534, -77.046774)"


In [41]:
full_merge['miles'] = full_merge.apply(lambda row: vincenty(row['lat_lon_start'], row['lat_lon_end']).miles, axis=1)

In [42]:
full_merge.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,atemp,hum,windspeed,start_lat,start_lon,end_lat,end_lon,lat_lon_start,lat_lon_end,miles
0,2011-03-31 23:58:00,2011-04-01 00:00:00,14th & Harvard St NW,31105,16th & Harvard St NW,31103,Registered,2011-03-31,2011-04-01,1.833,...,0.257575,0.918333,0.217646,38.926800,-77.032200,38.926088,-77.036536,"(38.9268, -77.0322)","(38.926088, -77.036536)",0.238743
2,2011-03-31 23:47:00,2011-03-31 23:50:00,Lincoln Park / 13th & East Capitol St NE,31619,13th & H St NE,31611,Registered,2011-03-31,2011-03-31,3.317,...,0.257575,0.918333,0.217646,38.890461,-76.988355,38.900022,-76.988422,"(38.890461, -76.988355)","(38.900022, -76.988422)",0.659533
3,2011-03-31 23:45:00,2011-03-31 23:50:00,14th & R St NW,31202,Massachusetts Ave & Dupont Circle NW,31200,Registered,2011-03-31,2011-03-31,5.733,...,0.257575,0.918333,0.217646,38.912939,-77.032085,38.910100,-77.044400,"(38.912939, -77.032085)","(38.9101, -77.0444)",0.691996
4,2011-03-31 23:34:00,2011-03-31 23:43:00,Park Rd & Holmead Pl NW,31602,15th & P St NW,31201,Registered,2011-03-31,2011-03-31,8.700,...,0.257575,0.918333,0.217646,38.930800,-77.031500,38.909850,-77.034438,"(38.9308, -77.0315)","(38.90985, -77.034438)",1.453796
5,2011-03-31 23:31:00,2011-03-31 23:39:00,37th & O St NW / Georgetown University,31236,21st & M St NW,31212,Registered,2011-03-31,2011-03-31,7.850,...,0.257575,0.918333,0.217646,38.907740,-77.071652,38.905340,-77.046774,"(38.90774, -77.071652)","(38.90534, -77.046774)",1.351053


In [43]:
full_merge.shape

(1219847, 27)

In [44]:
full_merge.to_csv('bikeshare_lat_lon.csv')

In [45]:
full_merge.drop('lat_lon_start', 1, inplace=True)

In [46]:
full_merge.drop('lat_lon_end', 1, inplace=True)

In [47]:
full_merge.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,weathersit,temp,atemp,hum,windspeed,start_lat,start_lon,end_lat,end_lon,miles
0,2011-03-31 23:58:00,2011-04-01 00:00:00,14th & Harvard St NW,31105,16th & Harvard St NW,31103,Registered,2011-03-31,2011-04-01,1.833,...,3,0.268333,0.257575,0.918333,0.217646,38.926800,-77.032200,38.926088,-77.036536,0.238743
2,2011-03-31 23:47:00,2011-03-31 23:50:00,Lincoln Park / 13th & East Capitol St NE,31619,13th & H St NE,31611,Registered,2011-03-31,2011-03-31,3.317,...,3,0.268333,0.257575,0.918333,0.217646,38.890461,-76.988355,38.900022,-76.988422,0.659533
3,2011-03-31 23:45:00,2011-03-31 23:50:00,14th & R St NW,31202,Massachusetts Ave & Dupont Circle NW,31200,Registered,2011-03-31,2011-03-31,5.733,...,3,0.268333,0.257575,0.918333,0.217646,38.912939,-77.032085,38.910100,-77.044400,0.691996
4,2011-03-31 23:34:00,2011-03-31 23:43:00,Park Rd & Holmead Pl NW,31602,15th & P St NW,31201,Registered,2011-03-31,2011-03-31,8.700,...,3,0.268333,0.257575,0.918333,0.217646,38.930800,-77.031500,38.909850,-77.034438,1.453796
5,2011-03-31 23:31:00,2011-03-31 23:39:00,37th & O St NW / Georgetown University,31236,21st & M St NW,31212,Registered,2011-03-31,2011-03-31,7.850,...,3,0.268333,0.257575,0.918333,0.217646,38.907740,-77.071652,38.905340,-77.046774,1.351053


In [48]:
full_merge.to_csv('smaller_bikeshare_lat_lon.csv')

In [49]:
full_merge.drop('atemp', 1, inplace=True)

In [50]:
full_merge.to_csv('less_bikeshare_lat_lon.csv')